In [1]:
import pandas as pd
import numpy as np
import joblib

## ML libraries
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from skopt import BayesSearchCV
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import log_loss, accuracy_score

## visalization libraries
import seaborn as sns
import plotly.express as px

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as mpatches
from matplotlib.ticker import FuncFormatter
from matplotlib.patches import Patch

pd.options.display.max_columns = None
pd.set_option('display.float_format', '{:2f}'.format)

## Data Prep

In [5]:
master_trackman = pd.read_csv('../../Trackman CSV Files/merged_trackman_games.csv')
master_trackman.head()

/var/folders/bp/l1vb_64s6tj40wftrxyjhnfm0000gn/T/ipykernel_15658/2981408051.py:1: DtypeWarning: Columns (91,92) have mixed types. Specify dtype option on import or set low_memory=False.
  master_trackman = pd.read_csv('../../Trackman CSV Files/merged_trackman_games.csv')


,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,Batter,BatterId,BatterSide,BatterTeam,PitcherSet,Inning,Top/Bottom,Outs,Balls,Strikes,TaggedPitchType,AutoPitchType,PitchCall,KorBB,TaggedHitType,PlayResult,OutsOnPlay,RunsScored,RelSpeed,VertRelAngle,HorzRelAngle,SpinRate,SpinAxis,Tilt,RelHeight,RelSide,Extension,VertBreak,InducedVertBreak,HorzBreak,PlateLocHeight,PlateLocSide,ZoneSpeed,VertApprAngle,HorzApprAngle,ZoneTime,ExitSpeed,Angle,Direction,HitSpinRate,PositionAt110X,PositionAt110Y,PositionAt110Z,Distance,LastTrackedDistance,Bearing,HangTime,pfxx,pfxz,x0,y0,z0,vx0,vy0,vz0,ax0,ay0,az0,HomeTeam,AwayTeam,Stadium,Level,League,GameID,PitchUID,EffectiveVelo,MaxHeight,MeasuredDuration,SpeedDrop,PitchLastMeasuredX,PitchLastMeasuredY,PitchLastMeasuredZ,ContactPositionX,ContactPositionY,ContactPositionZ,GameUID,UTCDate,UTCTime,LocalDateTime,UTCDateTime,AutoHitType,System,HomeTeamForeignID,AwayTeamForeignID,GameForeignID,Catcher,CatcherId,CatcherThrows,CatcherTeam,PlayID,PitchTrajectoryXc0,PitchTrajectoryXc1,PitchTrajectoryXc2,PitchTrajectoryYc0,PitchTrajectoryYc1,PitchTrajectoryYc2,PitchTrajectoryZc0,PitchTrajectoryZc1,PitchTrajectoryZc2,HitSpinAxis,HitTrajectoryXc0,HitTrajectoryXc1,HitTrajectoryXc2,HitTrajectoryXc3,HitTrajectoryXc4,HitTrajectoryXc5,HitTrajectoryXc6,HitTrajectoryXc7,HitTrajectoryXc8,HitTrajectoryYc0,HitTrajectoryYc1,HitTrajectoryYc2,HitTrajectoryYc3,HitTrajectoryYc4,HitTrajectoryYc5,HitTrajectoryYc6,HitTrajectoryYc7,HitTrajectoryYc8,HitTrajectoryZc0,HitTrajectoryZc1,HitTrajectoryZc2,HitTrajectoryZc3,HitTrajectoryZc4,HitTrajectoryZc5,HitTrajectoryZc6,HitTrajectoryZc7,HitTrajectoryZc8,ThrowSpeed,PopTime,ExchangeTime,TimeToBase,CatchPositionX,CatchPositionY,CatchPositionZ,ThrowPositionX,ThrowPositionY,ThrowPositionZ,BasePositionX,BasePositionY,BasePositionZ,ThrowTrajectoryXc0,ThrowTrajectoryXc1,ThrowTrajectoryXc2,ThrowTrajectoryYc0,ThrowTrajectoryYc1,ThrowTrajectoryYc2,ThrowTrajectoryZc0,ThrowTrajectoryZc1,ThrowTrajectoryZc2,PitchReleaseConfidence,PitchLocationConfidence,PitchMovementConfidence,HitLaunchConfidence,HitLandingConfidence,CatcherThrowCatchConfidence,CatcherThrowReleaseConfidence,CatcherThrowLocationConfidence
0,1,2025-04-08,19:04:21.64,1,1,"Gilbert, Bradley",831228.000000,Right,WAS_HUS,"Ishikawa, Keni",1000274623.000000,Left,SEA_RED,Undefined,1,Top,0,0,0,Fastball,Four-Seam,BallCalled,Undefined,Undefined,Undefined,0,0,90.028060,-3.296976,-1.238717,2497.731876,193.680625,12:30,5.621390,0.510940,5.836200,-16.090760,18.231070,4.190400,1.212030,-0.291350,83.110150,-6.326811,-0.487350,0.421654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.463980,10.486150,-0.412500,50.000000,5.342850,2.649920,-130.679410,-8.015750,-4.299720,25.181650,-13.875380,WAS_HUS,SEA_RED,UofWashington,D1,BIG10,20250408-UofWashington-1,18b54860-14e7-11f0-938f-b5417cf72451,89.391770,NaN,NaN,6.917910,NaN,NaN,NaN,NaN,NaN,NaN,6547872e-3314-45b6-a0d2-1680b11cc799,2025-04-09,02:04:21.64,2025-04-08T19:04:21.6401920-07:00,2025-04-09T02:04:21.6401920Z,NaN,v3,471486.000000,4504,08 April 2025 21:05-seattleu@washington,"Blanchard, Colin",1000114026.000000,Right,WAS_HUS,69bcc3f1-572b-4f97-8e16-e8d65a83174d,54.662890,-131.574870,12.590830,5.619120,-7.522340,-6.937690,-0.509450,2.802820,-2.149860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
1,2,2025-04-08,19:04:34.59,1,2,"Gilbert, Bradley",831228.000000,Right,WAS_HUS,"Ishikawa, Keni",1000274623.000000,Left,SEA_RED,Undefined,1,Top,0,1,0,Slider,Slider,BallCalled,Undefined,Undefined,Undefined,0,0,82.167840,1.675543,-0.415000,2782.722426,155.127014,11:15,5.533740,0.741550,4.993150,-35.983520,6.645160,-2.487760,4.117590,0.142400,75.363000,-4.887176,-0.853995,0.469918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.118030,3.249250,-0.699510,50.000000,5.665960,0.971940,-119.219350,2.168950

In [9]:
necessary_data = master_trackman[['Pitcher', 'PitcherThrows', 'BatterSide', 'TaggedPitchType', 'AutoPitchType', 'PitchCall', 'KorBB', 'TaggedHitType', 'PlayResult', 'OutsOnPlay', 'RunsScored', 'RelSpeed', 'SpinRate', 'Tilt', 'RelHeight', 'RelSide', 'Extension', 'InducedVertBreak', 'HorzBreak', 'PlateLocHeight', 'PlateLocSide', 'ExitSpeed', 'Angle', 'Direction', 'Distance', 'Bearing', 'HangTime', 'PitchUID']]
necessary_data = necessary_data.dropna(subset = ['ExitSpeed', 'Angle', 'TaggedHitType', 'PitchCall', 'PlayResult'])
necessary_data

,Pitcher,PitcherThrows,BatterSide,TaggedPitchType,AutoPitchType,PitchCall,KorBB,TaggedHitType,PlayResult,OutsOnPlay,RunsScored,RelSpeed,SpinRate,Tilt,RelHeight,RelSide,Extension,InducedVertBreak,HorzBreak,PlateLocHeight,PlateLocSide,ExitSpeed,Angle,Direction,Distance,Bearing,HangTime,PitchUID
10,"Gilbert, Bradley",Right,Right,Slider,Slider,InPlay,Undefined,FlyBall,Out,1,0,82.470560,2725.977171,9:30,5.546190,0.733100,5.050820,2.953940,-4.669880,1.773770,-0.071790,90.936520,40.323030,-5.968970,276.412990,-4.938970,5.767655,ac9ce650-14e7-11f0-938f-b5417cf72451
14,"Gilbert, Bradley",Right,Left,Fastball,Four-Seam,FoulBallNotFieldable,Undefined,Undefined,Undefined,0,0,89.312210,2529.515979,12:30,5.658450,0.803300,5.777000,18.854700,3.916420,3.943280,0.049260,71.937640,24.785963,-171.534257,182.529480,-169.338207,2.222835,efd3c740-14e7-11f0-938f-b5417cf72451
15,"Gilbert, Bradley",Right,Left,Fastball,Four-Seam,FoulBallNotFieldable,Undefined,Undefined,Undefined,0,0,88.662170,2431.139672,12:30,5.677130,0.634130,6.012300,22.250840,5.059140,2.907520,-0.684900,94.798200,63.631363,58.530611,NaN,NaN,NaN,009d1c70-14e8-11f0-938f-b5417cf72451
17,"Gilbert, Bradley",Right,Left,Fastball,Changeup,InPlay,Undefined,LineDrive,Single,0,1,86.708720,1876.123399,2:30,5.507600,0.900390,5.883020,7.790750,21.118200,2.254870,1.476370,69.382380,22.578153,-8.859449,213.886670,-17.805961,3.209279,24846530-14e8-11f0-938f-b5417cf72451
22,"Gilbert, Bradley",Right,Right,Fastball,Four-Seam,FoulBallNotFieldable,Undefined,Undefined,Undefined,0,0,88.191290,2456.377665,12:45,5.751090,0.489740,5.863180,18.733590,6.290440,2.189860,-0.766800,72.415980,12.091078,170.800229,113.651290,173.585101,1.260415,9c699a20-14e8-11f0-938f-b5417cf72451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1354505,"Hicks, Garrett",Right,Left,Cutter,Slider,FoulBallNotFieldable,Undefined,Undefined,Undefined,0,0,85.553090,2227.931728,11:30,5.344160,2.339540,5.755360,7.349650,-2.008900,3.119210,-0.565340,64.467140,62.465453,-45.576647,116.290870,-56.197124,4.700993,298a7a20-11d6-11f0-b75a-15defc8841bc
1354506,"Hicks, Garrett",Right,Left,Fastball,Four-Seam,FoulBallNotFieldable,Undefined,Undefined,Undefined,0,0,93.261950,2178.447299,1:00,5.402980,2.212650,6.065110,18.922960,10.575950,3.168300,0.596340,73.412110,65.922206,-82.634590,NaN,NaN,NaN,3c7a31c0-11d6-11f0-b75a-15defc8841bc
1354509,"Hicks, Garrett",Right,Right,Cutter,Slider,FoulBallNotFieldable,Undefined,Undefined,Undefined,0,0,87.422390,2299.056403,10:45,5.260250,2.269660,6.038550,6.124730,-3.600440,1.818140,0.106260,46.703320,50.600354,120.524254,116.274000,126.704006,3.081668,6451ee90-11d6-11f0-b75a-15defc8841bc
1354510,"Hicks, Garrett",Right,Right,Cutter,Slider,InPlay,Undefined,Popup,Out,1,0,87.589110,2338.856413,10:30,5.275490,2.220830,6.206060,4.687760,-3.380080,2.490030,-0.199540,68.609820,40.223331,-3.564189,197.333360,8.108297,4.536495,6f267430-11d6-11f0-b75a-15defc8841bc
